<a href="https://colab.research.google.com/github/aggelospsiris/Book-ratings-guess-using-kmeans-and-neural-networks/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive 
drive.mount('/drive')
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


Data preparation

In [3]:
df=pd.read_csv('/drive/My Drive/Colab Notebooks/info_retrival/train_after_pca_titles_summaries.csv')
#drop unusuful column
df = df.drop(columns = ['Unnamed: 0'])
array = df.to_numpy()
# Split into input and output
X = array[:,1 :-1]
Y = array[:, -1]
# Features normalization
Y = to_categorical(Y)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,shuffle=True,stratify=Y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2,shuffle=True,stratify=y_train)

In [4]:
#the x train data contains the cluster,the summaries and the titles
X.shape

(22470, 4)

In [5]:
#the target data contains the ratings(0-10) into a categorical format
Y.shape

(22470, 11)

Built and compile lstm model

In [31]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(X.shape[1],1)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,activation='relu', return_sequences=True,kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,activation='relu',kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(tf.keras.layers.Dense(units=Y.shape[1], activation='softmax'))

model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=['accuracy'])

callback = EarlyStopping(monitor="val_loss", mode="max", min_delta=0, patience=20, verbose=0)

From the following results i see that the accuracy is stack  on local maximum.The ways i tried to fix this problem is :


 *  Use statify in train_test_split()  
 *  Use different optimizer with different lr and momentum
 *  Change the structure of my model into more complex one in a case i have the underfitting problem
 *  Expirements with adding or removeing dropout
 * Change the batch size
 * Use weight decay 

 Non of them work so the problem that the any model can capture the sequence of the data cause they are unpredictable.








In [32]:
model.fit(x_train, y_train, batch_size=128, callbacks=[callback],epochs=200, validation_data=(x_val, y_val))

Epoch 1/200
113/113 [==============================] - 11s 57ms/step - loss: 7.9178 - accuracy: 0.3927 - val_loss: 5.0997 - val_accuracy: 0.3954
Epoch 2/200
113/113 [==============================] - 6s 50ms/step - loss: 4.6109 - accuracy: 0.3953 - val_loss: 4.2294 - val_accuracy: 0.3954
Epoch 3/200
113/113 [==============================] - 6s 50ms/step - loss: 3.9385 - accuracy: 0.3953 - val_loss: 3.6590 - val_accuracy: 0.3954
Epoch 4/200
113/113 [==============================] - 6s 50ms/step - loss: 3.4293 - accuracy: 0.3953 - val_loss: 3.2060 - val_accuracy: 0.3954
Epoch 5/200
113/113 [==============================] - 6s 50ms/step - loss: 3.0227 - accuracy: 0.3953 - val_loss: 2.8443 - val_accuracy: 0.3954
Epoch 6/200
113/113 [==============================] - 6s 50ms/step - loss: 2.7012 - accuracy: 0.3953 - val_loss: 2.5628 - val_accuracy: 0.3954
Epoch 7/200
113/113 [==============================] - 6s 50ms/step - loss: 2.4531 - accuracy: 0.3953 - val_loss: 2.3466 - val_accuracy

In [22]:
metrics = model.evaluate(x_test,y_test,verbose=1)

141/141 [==============================] - 1s 9ms/step - loss: 1.8053 - accuracy: 0.3954
